In [1]:
import pandas as pd
from pathlib import Path
import IMP
import IMP.atom
import IMP.rmf
import RMF
import random
import numpy as np
import matplotlib.pyplot as plt

import IMP
import IMP.core
import IMP.atom
import IMP.container
import IMP.isd
import IMP.pmi.tools
from operator import itemgetter
import math

import time

In [2]:
gsms_dir = Path(Path.home(), "Documents/mtorc2/tmp/1000R")


In [3]:
# sampcon_job_dir = Path("/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/3")

# # gsms_dir = Path("/Users/matthew/Downloads/116/1000_edit")
# # sampcon_job_dir = Path("/Users/matthew/Downloads/116/3")

# cluster = 0
# cluster_dir = Path(sampcon_job_dir, "cluster.{}".format(cluster))

# # Script needs to read in the number of structures contained in the A.rmf3 file because the structures in sampcon cluster A are indexed between 0:N_A-1 while structures in cluster B are indexed between N_A:N_A+N_B-1.
# with open(Path(gsms_dir, "A.txt"), 'r') as fp:
#     N_A = len(fp.readlines())
#     print("N_A: {}".format(N_A))

# samp_clust_file = Path(sampcon_job_dir, "cluster.{}.all.txt".format(cluster))
# samp_clust_df = pd.read_csv(samp_clust_file, header=None)
# # samp_clust_df = samp_clust_df.iloc[:1]

# print(samp_clust_file)
# print(len(samp_clust_df))

# models = list()
# for i in range(len(samp_clust_df)):
#     frame_num = samp_clust_df.iloc[i,0]
#     if frame_num < N_A:
#         rmf_file = Path(gsms_dir, "A.rmf3")
#     else:
#         rmf_file = Path(gsms_dir, "B.rmf3")
#         frame_num = frame_num - N_A

#     models.append((rmf_file, frame_num))



In [3]:
included_objects = list()

rmf_file = Path(gsms_dir, "A.rmf3")
for i in range(1000):
    fh = RMF.open_rmf_file_read_only(str(rmf_file))
    m = IMP.Model()
    h = IMP.rmf.create_hierarchies(fh, m)[0]
    IMP.rmf.load_frame(fh, RMF.FrameID(i))

    pids = IMP.atom.Selection(h, resolution=10).get_selected_particle_indexes()

    print(len(pids))

    # n_clash = 0
    # for pid_0 in pids:
    #     xyzr_0  = IMP.core.XYZR(m, pid_0)
    #     for pid_1 in pids:
    #         xyzr_1  = IMP.core.XYZR(m, pid_1)

    #         if pid_0 != pid_1 and IMP.core.get_distance(xyzr_0, xyzr_1) < xyzr_0.get_radius() + xyzr_1.get_radius():
    #             print("Overlap ", pid_0, pid_1)

    #             n_clash += 1

        # break



    break

1166


In [16]:
n_clash / (len(pids) * (len(pids) - 1))

0.014262472485810408

In [7]:
kappa = 1
cpc = None

# gather IMP hierarchies from input objects
hierarchies = IMP.pmi.tools.input_adaptor(included_objects,
                                          10,
                                          flatten=True)
included_ps = []

# perform selection
mdl = hierarchies[0].get_model()

included_ps = [h.get_particle() for h in hierarchies]

# setup score
ssps = IMP.core.SoftSpherePairScore(kappa)
lsa = IMP.container.ListSingletonContainer(m)
lsa.add(IMP.get_indexes(included_ps))

# setup close pair container
rbcpf = IMP.core.RigidClosePairsFinder()
cpc = IMP.container.ClosePairContainer(lsa, 0.0, rbcpf, 10.0)
evr = IMP.container.PairsRestraint(ssps, cpc)
evr.evaluate(False)

5.140711380565968

In [ ]:
cpc.get_indexes()

In [ ]:
kappa = 1
close_pairs = cpc.get_indexes()
for i in range(len(close_pairs)):
    pt_0 = m.get_particle(close_pairs[i][0])
    pt_1 = m.get_particle(close_pairs[i][1])

    hu = IMP.core.HarmonicLowerBound(0, kappa)

    dps = IMP.core.SphereDistancePairScore(hu)
#                 sphere_dist = math.sqrt(dps.evaluate_index(m, (pt0.get_index(), pt1.get_index()), None) * 2 / 10)

    xyz_0 = IMP.core.XYZR(pt_0)
    xyz_1 = IMP.core.XYZR(pt_1)

#                 r_0 = xyz_0.get_radius()
#                 r_1 = xyz_1.get_radius()

#                 d = IMP.algebra.distance(xyz_0,xyz_1)

    print(IMP.algebra.get_distance(xyz_0.get_sphere(), xyz_1.get_sphere()))


In [ ]:
ev_dict = dict()
for rmf_file, frame_num in models:
    print(rmf_file, frame_num)
    included_objects = list()
    fh = RMF.open_rmf_file_read_only(str(rmf_file))
    m = IMP.Model()
    h = IMP.rmf.create_hierarchies(fh, m)[0]
    IMP.rmf.load_frame(fh, RMF.FrameID(frame_num))
    for i in range(0,10,2):
        h_mol = h.get_children()[0].get_children()[i]
        included_objects.append(h_mol)

    # gather IMP hierarchies from input objects
    hierarchies = IMP.pmi.tools.input_adaptor(included_objects,
                                              10,
                                              flatten=True)
    included_ps = []

    # perform selection
    mdl = hierarchies[0].get_model()

    included_ps = [h.get_particle() for h in hierarchies]

    # setup score
    ssps = IMP.core.SoftSpherePairScore(kappa)
    lsa = IMP.container.ListSingletonContainer(m)
    lsa.add(IMP.get_indexes(included_ps))

    # setup close pair container
    rbcpf = IMP.core.RigidClosePairsFinder()
    cpc = IMP.container.ClosePairContainer(lsa, 0.0, rbcpf, 10.0)
    evr = IMP.container.PairsRestraint(ssps, cpc)
    evr.evaluate(False)
    close_pairs = cpc.get_indexes()
    for j in range(len(close_pairs)):
        pt_0 = m.get_particle(close_pairs[j][0])
        pt_1 = m.get_particle(close_pairs[j][1])

        entry = "{},{}".format(str(close_pairs[j][0]), str(close_pairs[j][1]))

        hu = IMP.core.HarmonicLowerBound(0, kappa)

        dps = IMP.core.SphereDistancePairScore(hu)
    #                 sphere_dist = math.sqrt(dps.evaluate_index(m, (pt0.get_index(), pt1.get_index()), None) * 2 / 10)

        xyz_0 = IMP.core.XYZR(pt_0)
        xyz_1 = IMP.core.XYZR(pt_1)

    #                 r_0 = xyz_0.get_radius()
    #                 r_1 = xyz_1.get_radius()

    #                 d = IMP.algebra.distance(xyz_0,xyz_1)

        d = IMP.algebra.get_distance(xyz_0.get_sphere(), xyz_1.get_sphere())

        if entry in ev_dict.keys():
            ev_dict[entry].append(d)
        else:
            ev_dict[entry] = [d]



In [5]:
rmf_file, frame_num = models[0]
fh = RMF.open_rmf_file_read_only(str(rmf_file))
m = IMP.Model()
h = IMP.rmf.create_hierarchies(fh, m)[0]
IMP.rmf.load_frame(fh, RMF.FrameID(frame_num))
for i in range(0,10,2):
    h_mol = h.get_children()[0].get_children()[i]
    included_objects.append(h_mol)

hierarchies = IMP.pmi.tools.input_adaptor(included_objects,
                                          10,
                                          flatten=True)
mdl = hierarchies[0].get_model()

included_ps = [h.get_particle() for h in hierarchies]

ev_matrix = np.ndarray(shape=(len(included_ps),len(included_ps),len(models)))


In [6]:

kappa = 10

ev_dict = dict()
# for rmf_file, frame_num in models[:10]:
# for i in range(len(models)):
for i in range(10):
    t0 = time.time()
    rmf_file, frame_num = models[i]
    print(rmf_file, frame_num)
    included_objects = list()
    fh = RMF.open_rmf_file_read_only(str(rmf_file))
    m = IMP.Model()
    h = IMP.rmf.create_hierarchies(fh, m)[0]
    IMP.rmf.load_frame(fh, RMF.FrameID(frame_num))
    for i in range(0,10,2):
        h_mol = h.get_children()[0].get_children()[i]
        included_objects.append(h_mol)

    hierarchies = IMP.pmi.tools.input_adaptor(included_objects,
                                              10,
                                              flatten=True)
    mdl = hierarchies[0].get_model()
    included_ps = [h.get_particle() for h in hierarchies]

    for j in range(len(included_ps)):
        for k in range(len(included_ps)):
            pt_0 = included_ps[i]
            pt_1 = included_ps[j]

            xyz_0 = IMP.core.XYZR(pt_0)
            xyz_1 = IMP.core.XYZR(pt_1)
            d = IMP.algebra.get_distance(xyz_0.get_sphere(), xyz_1.get_sphere())
            ev_matrix[j,k,i] = d
            ev_matrix[k,j,i] = d
    t1 = time.time()
    print(t0 - t1)


/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit/A.rmf3 0
-10.476304769515991
/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit/A.rmf3 1
-8.984430313110352
/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit/A.rmf3 2
-8.736343145370483
/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit/A.rmf3 3
-8.803475141525269
/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit/A.rmf3 4
-8.689037084579468
/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit/A.rmf3 5
-8.702443361282349
/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit/A.rmf3 6
-8.698261976242065
/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit/A.rmf3 7
-8.810563087463379
/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/10

In [60]:
print(included_ps[477:492])

["1", "Fragment [2-12)", "Fragment [12-22)", "Fragment [22-32)", "Fragment [32-42)", "Fragment [42-50)", "50-59_bead", "60-69_bead", "70-76_bead", "Fragment [77-87)", "Fragment [87-97)", "Fragment [97-107)", "Fragment [107-117)", "Fragment [117-127)", "Fragment [127-133)"]


In [27]:
ranges_dict = {}
ranges_dict["MTOR"] = range(267)
ranges_dict["RICTOR"] = range(267,443)
ranges_dict["MLST8"] = range(443,477)
ranges_dict["MSIN1_0"] = range(477,492)
ranges_dict["MSIN1_1"] = range(492,536)
ranges_dict["AKT1"] = range(536,589)

In [18]:
len(ranges_dict["MTOR"])  * 589

157263

In [28]:
for mol in ranges_dict.keys():
    print(mol, list(ranges_dict[mol]))

MTOR [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,

In [35]:
def non_motive_ev_pairs(
    mol,
    bead2_mols
):
    mol = "MTOR"
    ev_pairs = list()
    for bead1_id in ranges_dict[mol]:
        bead1 = included_ps[bead1_id]
        bead1_name = str(bead1).split("\"")[1]

        bead2_list = list()
        if "bead" in bead1_name:
            for bead2_mol in bead2_mols:
#                 print(bead2_mol, list(ranges_dict[bead2_mol]))
                bead2_list.extend(list(ranges_dict[bead2_mol]))
        else:
            for bead2_mol in list(set(["AKT1", "MSIN1_1"])-set([mol])):
                bead2_list.extend(list(ranges_dict[bead2_mol]))

#         print(bead1, len(bead2_list), bead2_list)
        for bead2_id in bead2_list:
            if bead2_id != bead1_id:
                ev_pairs.append((bead1_id, bead2_id))

    return ev_pairs


In [54]:
import itertools
bead1_ids = list(range(588))
bead2_ids = list(range(588))
ev_pairs = list(itertools.product(bead1_ids, bead2_ids))
print(len(ev_pairs))


345744


In [66]:
## Remove duplicates and self pairs
ev_pairs_filter = set(ev_pairs.copy())
for ev_pair in ev_pairs:
    ev_pair_dup = (ev_pair[1], ev_pair[0])
    if ev_pair in ev_pairs_filter and ev_pair_dup in ev_pairs_filter:
        ev_pairs_filter.remove(ev_pair_dup)

len(ev_pairs_filter)

172578

In [ ]:
## Remove mTOR, Rictor, mLST8, mSin1 rigid body rigid body pairs
rb_rb_pairs = list()
for mol1 in ["MTOR", "RICTOR", "MLST8", "MSIN1_0"]:
    for mol2 in ["MTOR", "RICTOR", "MLST8", "MSIN1_0"]:
        for bead1_id in ranges_dict[mol]:
            for bead2_id in ranges_dict[mol]:
                bead1 = included_ps[bead1_id]
                bead2 = included_ps[bead2_id]


#             bead1_name = str(bead1).split("\"")[1]

#             bead2_list = list()
#             if "bead" in bead1_name:
#                 for bead2_mol in bead2_mols:
#         #                 print(bead2_mol, list(ranges_dict[bead2_mol]))
#                     bead2_list.extend(list(ranges_dict[bead2_mol]))
#             else:
#                 for bead2_mol in list(set(["AKT1", "MSIN1_1"])-set([mol])):
#                     bead2_list.extend(list(ranges_dict[bead2_mol]))

#         #         print(bead1, len(bead2_list), bead2_list)
#             for bead2_id in bead2_list:
#                 if bead2_id != bead1_id:
#                     ev_pairs.append((bead1_id, bead2_id))

#         return ev_pairs


In [36]:
len(non_motive_ev_pairs("MTOR", ["MTOR", "RICTOR", "MLST8", "MSIN1_0", "MSIN1_1", "AKT1"]))


45048

In [31]:
len(non_motive_ev_pairs("RICTOR", ["RICTOR", "MLST8", "MSIN1_0", "MSIN1_1", "AKT1"]))


RICTOR [267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442]
MLST8 [443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 

322

In [16]:
len(non_motive_ev_pairs("MLST8", ["MLST8", "MSIN1_0", "MSIN1_1", "AKT1"]))


162792

In [68]:
set(ranges_dict.keys()) - set("MTOR")

{'AKT1', 'MLST8', 'MSIN1_0', 'MSIN1_1', 'MTOR', 'RICTOR'}

In [63]:
len(ev_pairs)

22932

In [46]:
len(included_ps)

589

In [ ]:
m.get_particle_name(pt_1)

In [ ]:
IMP.atom.Domain(pt_0)

In [ ]:
list(ev_dict.keys())[0]

In [ ]:
589 * 589

In [ ]:
print(len(ev_dict))

In [ ]:
for key in ev_dict.keys():
    print(len(ev_dict[key]))

In [ ]:
print(len(ev_dict.keys()))

In [ ]:
ev_df = pd.DataFrame(ev_dict)
ev_df.to_csv(Path(Path.home(), "mtorc2/manuscript/submission/models/116_693502_3/cluster.0/ev.csv"))

In [ ]:
rbcpf.get_close_pairs()

In [ ]:
"{},{}".format(str(close_pairs[0][0]), str(close_pairs[0][1]))

In [ ]:
cpc.get_indexes()

In [ ]:
evr.evaluate(False)

In [68]:
std_dev = -math.sqrt(1.380649 * 10**(-23) * 310 / 1) * 10**10


In [69]:
std_dev * 3

-1.9626539965057521

In [70]:
dist_matrix_merge = np.load(Path(Path.home(), "mtorc2/manuscript/submission/models/116_693502_3/cluster.0/distances.npy"))
ev_sat = np.where(dist_matrix_merge > 3 * std_dev, 0, 1)




In [71]:
unsat = 0
total = 0
for pair in ev_pairs_filter:
    if np.sum(ev_sat[pair[0],pair[1],:]) > 0:
        unsat = unsat + 1
        print(pair[0],pair[1],np.sum(ev_sat[pair[0],pair[1],:]))
    total = total + 1

542 544 1923
104 139 1923
449 451 1923
209 210 1923
304 341 1923
389 402 1
472 474 1923
142 143 1923
230 234 1923
345 348 1923
111 115 1923
212 224 1923
367 412 1923
425 426 1
305 356 1923
454 455 1923
445 474 1923
223 227 1923
541 581 1
431 435 1923
566 567 1923
127 130 1923
213 223 1923
462 464 1923
103 104 1923
574 575 1923
20 24 1923
569 573 1923
444 446 1923
158 159 1923
395 397 1
496 506 1
470 472 1923
496 498 1923
140 141 1923
218 292 1923
447 449 1923
465 470 1923
525 527 1923
135 139 13
270 271 1923
151 152 1923
218 293 1923
213 217 1923
94 98 1923
169 173 1923
120 124 1923
273 277 1923
448 475 1923
177 179 1923
565 570 1923
512 513 1923
538 539 1923
541 544 1923
185 187 1923
21 24 1923
564 565 1923
523 524 1
359 361 1923
362 366 1923
290 478 1923
460 462 1923
226 229 1923
430 439 1923
310 315 1923
60 61 1923
445 447 1923
557 558 1923
50 56 1923
364 413 1923
253 261 1923
471 473 1923
63 64 1
107 111 1923
572 574 1923
453 455 1923
24 31 1923
273 483 1
227 228 1923
175 176 1923


In [72]:
print(unsat, total)

326 172578


In [29]:
unsat = 0
total = 0
for i in range(ev_sat.shape[0]):
    for j in range(i+1, ev_sat.shape[1]):
        if np.sum(ev_sat[i,j,:]) > 0:
            unsat = unsat + 1
            print(i,j,np.sum(ev_sat[i,j,:]))
        total = total + 1


1 5 1
9 11 1923
11 15 1923
12 13 1923
14 15 1923
14 19 1923
18 19 1923
19 23 1923
20 21 1923
20 24 1923
21 24 1923
22 23 1923
23 30 1923
24 31 1923
30 45 1923
31 46 1923
41 46 1923
44 45 1
47 48 1
49 50 1923
50 56 1923
51 57 1923
52 55 1
53 54 1
56 59 1923
58 61 1923
60 61 1923
63 64 1
67 71 1923
70 71 1923
73 74 1
78 81 1923
79 80 1923
80 85 1923
81 82 1923
83 84 1923
86 90 1923
87 89 1923
89 93 1923
90 94 1923
91 94 1923
92 93 1923
93 97 1923
94 98 1923
98 169 1923
103 104 1923
103 146 1923
104 139 1923
105 106 1923
105 109 1923
107 111 1923
111 115 1923
112 116 1923
114 118 1923
116 120 1923
119 123 1923
120 124 1923
121 125 1923
123 127 1923
127 128 1923
127 130 1923
135 139 13
137 138 1923
137 144 1923
138 141 1923
139 140 1923
140 141 1923
142 143 1923
144 145 1923
144 148 1923
148 149 1923
149 151 1923
150 151 1923
151 152 1923
153 154 1923
155 158 1923
156 157 1923
158 159 1923
158 161 1923
159 161 1923
161 163 1923
163 164 1923
164 166 1923
167 168 1923
169 173 1923
170 171 19

In [25]:
dist_matrix_merge[0,1,:]

array([ 1.07070866,  1.90334214,  2.05452072, ...,  0.37641127,
       -0.13590734,  1.28665121])

In [30]:
(total - unsat) / total

0.9981174133490408